In [ ]:
#from gwlens.pe.waveform import lensed_lal_binary_black_hole

In [2]:
import numpy as np
frequency_array = np.linspace(20,100,1000)
mass_1 = 30
mass_2 = 30
luminosity_distance =  1000
a_1 = 0
tilt_1 = 0.1
phi_12 = 0.4 
a_2 = 0
tilt_2 = 0.1
phi_jl = 0.6
theta_jn = 0.8
phase = 0.5
MLz = 30
y = 0.1




In [4]:
from bilby.gw.source import lal_binary_black_hole
from scipy.constants import c,G
import numpy as np
import h5py
from scipy.interpolate import RectBivariateSpline
Msun = 1.988e+30
constant = (8*np.pi*G/c**3)*Msun


#load the interpolation file

file = "/work/sreekanth/codes/globular_cluster/ampfac/pml/outdir/interpolation/result_point.hdf5"
with h5py.File(file, 'r') as file:
    w_values = file['w_values'][:]
    y_values = file['y_values'][:]
    data = file['results'][:]  # (len(w), len(y), 2)

real_data = data[:, :, 0]
imag_data = data[:, :, 1]

real_spline = RectBivariateSpline(w_values,y_values, real_data, kx=3, ky=3)
imag_spline = RectBivariateSpline(w_values, y_values, imag_data, kx=3, ky=3)


def lensed_lal_binary_black_hole(frequency_array, mass_1, mass_2, luminosity_distance, a_1,tilt_1,
                                 phi_12,a_2,tilt_2,phi_jl,theta_jn,phase,MLz,y,**kwargs):
    
    ws = constant*MLz*frequency_array
    #F_interpolate = kwargs['F_amp']
    F = real_spline(ws,y) + 1j*imag_spline(ws,y)
    

    waveform_kwargs = dict(
        waveform_approximant='IMRPhenomXAS', reference_frequency=50.0,
        minimum_frequency=20.0, maximum_frequency=frequency_array[-1],
        catch_waveform_errors=False, pn_spin_order=-1, pn_tidal_order=-1,
        pn_phase_order=-1, pn_amplitude_order=0)
    waveform_kwargs.update(kwargs)
    waveform_dict = lal_binary_black_hole(
        frequency_array=frequency_array, mass_1=mass_1, mass_2=mass_2,
        luminosity_distance=luminosity_distance, theta_jn=theta_jn, phase=phase,
        a_1=a_1, a_2=a_2, tilt_1=tilt_1, tilt_2=tilt_2, phi_12=phi_12,
        phi_jl=phi_jl, **waveform_kwargs)
    waveform_dict["plus"] *= F
    waveform_dict["cross"] *= F
    return waveform_dict


In [17]:
ws = np.linspace(20,30,3)
y = 0.1
F = real_spline(ws,y,grid= True)[:,0] + 1j*imag_spline(ws,y,grid= True)[:,0]
F

array([0.81633255+1.58544018j, 0.28345129-0.39845172j,
       1.66468845-1.91576729j])

In [5]:
lensed_lal_binary_black_hole(frequency_array,mass_1,mass_2,luminosity_distance, a_1,tilt_1,
                                 phi_12,a_2,tilt_2,phi_jl,theta_jn,phase,MLz,y)

/work/sreekanth/miniconda3/envs/gwlenstest/lib/python3.11/site-packages/lalsimulation/lalsimulation.py:8: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


ValueError: non-broadcastable output operand with shape (1000,) doesn't match the broadcast shape (1000,1000)